In [14]:
import cv2
import os, sys, gc
import time
import numpy as np
import mediapipe as mp
from tqdm.auto import tqdm
import multiprocessing
from joblib import Parallel, delayed
from natsort import natsorted
from glob import glob
import math
import pickle


In [15]:
mp_holistic = mp.solutions.holistic

N_FACE_LANDMARKS = 468
N_BODY_LANDMARKS = 33
N_HAND_LANDMARKS = 21

In [16]:
class Counter(object):
    # https://stackoverflow.com/a/47562583/
    def __init__(self, initval=0):
        self.val = multiprocessing.RawValue("i", initval)
        self.lock = multiprocessing.Lock()

    def increment(self):
        with self.lock:
            self.val.value += 1

    @property
    def value(self):
        return self.val.value


In [17]:
def process_body_landmarks(component, n_points):
    kps = np.zeros((n_points, 3))
    conf = np.zeros(n_points)
    if component is not None:
        landmarks = component.landmark
        kps = np.array([[p.x, p.y, p.z] for p in landmarks])
        conf = np.array([p.visibility for p in landmarks])
    return kps, conf

In [18]:
def process_other_landmarks(component, n_points):
    kps = np.zeros((n_points, 3))
    conf = np.zeros(n_points)
    if component is not None:
        landmarks = component.landmark
        kps = np.array([[p.x, p.y, p.z] for p in landmarks])
        conf = np.ones(n_points)
    return kps, conf

In [36]:
def get_holistic_keypoints(frames):
    """
    Create the holistic object inside this function to avoid PicklingError.
    """
    holistic = mp_holistic.Holistic(static_image_mode=False, model_complexity=2)
    keypoints = []
    confs = []

    for frame in frames:
        results = holistic.process(frame)

        body_data, body_conf = process_body_landmarks(
            results.pose_landmarks, N_BODY_LANDMARKS
        )
        face_data, face_conf = process_other_landmarks(
            results.face_landmarks, N_FACE_LANDMARKS
        )
        lh_data, lh_conf = process_other_landmarks(
            results.left_hand_landmarks, N_HAND_LANDMARKS
        )
        rh_data, rh_conf = process_other_landmarks(
            results.right_hand_landmarks, N_HAND_LANDMARKS
        )

        data = np.concatenate([body_data, face_data, lh_data, rh_data])
        conf = np.concatenate([body_conf, face_conf, lh_conf, rh_conf])

        keypoints.append(data)
        confs.append(conf)

    # Close holistic object after processing
    holistic.close()
    gc.collect()

    keypoints = np.stack(keypoints)
    confs = np.stack(confs)
    return keypoints, confs


In [20]:
def gen_keypoints_for_frames(frames, save_path):

    pose_kps, pose_confs = get_holistic_keypoints(frames)
    body_kps = np.concatenate([pose_kps[:, :33, :], pose_kps[:, 501:, :]], axis=1)

    confs = np.concatenate([pose_confs[:, :33], pose_confs[:, 501:]], axis=1)

    d = {"keypoints": body_kps, "confidences": confs}

    with open(save_path + ".pkl", "wb") as f:
        pickle.dump(d, f, protocol=4)

In [25]:
def load_frames_from_video(video_path):
    frames = []
    vidcap = cv2.VideoCapture(video_path)
    while vidcap.isOpened():
        success, img = vidcap.read()
        if not success:
            break
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames.append(img)
    vidcap.release()
    print(f"Loaded {len(frames)} frames from {video_path}")
    return np.asarray(frames)


In [22]:
def load_frames_from_folder(frames_folder, patterns=["*.jpg"]):
    images = []
    for pattern in patterns:
        images.extend(glob(f"{frames_folder}/{pattern}"))
    images = natsorted(list(set(images)))  # remove dupes
    if not images:
        exit(f"ERROR: No frames in folder: {frames_folder}")

    frames = []
    for img_path in images:
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        frames.append(img)

    return np.asarray(frames)

W0000 00:00:1727954269.424566 4058347 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727954269.428579 4058361 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727954269.430475 4058358 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727954269.430726 4058356 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727954269.445292 4058360 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727954269.450455 4058362 inference_feedback_manager.cc:114] Feedback manager 

In [23]:
def gen_keypoints_for_video(video_path, save_path):
    if not os.path.isfile(video_path):
        print("SKIPPING MISSING FILE:", video_path)
        return
    frames = load_frames_from_video(video_path)
    gen_keypoints_for_frames(frames, save_path)


def gen_keypoints_for_folder(folder, save_path, file_patterns):
    frames = load_frames_from_folder(folder, file_patterns)
    gen_keypoints_for_frames(frames, save_path)


W0000 00:00:1727954269.466792 4058359 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [24]:
def generate_pose(dataset, save_folder, worker_index, num_workers, counter):
    num_splits = math.ceil(len(dataset) / num_workers)
    end_index = min((worker_index + 1) * num_splits, len(dataset))
    for index in range(worker_index * num_splits, end_index):
        imgs, label, video_id = dataset.read_data(index)
        save_path = os.path.join(save_folder, video_id)
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        gen_keypoints_for_frames(imgs, save_path)
        counter.increment()


In [13]:
def dump_pose_for_dataset(
    dataset, save_folder, num_workers=multiprocessing.cpu_count()
):
    os.makedirs(save_folder, exist_ok=True)
    processes = []
    counter = Counter()
    for i in tqdm(range(num_workers), desc="Creating sub-processes..."):
        p = multiprocessing.Process(
            target=generate_pose, args=(dataset, save_folder, i, num_workers, counter)
        )
        p.start()
        processes.append(p)

    total_samples = len(dataset)
    with tqdm(total=total_samples) as pbar:
        while counter.value < total_samples:
            pbar.update(counter.value - pbar.n)
            time.sleep(2)

    for i in range(num_workers):
        processes[i].join()
    print(f"Pose data successfully saved to: {save_folder}")

In [37]:
if __name__ == "__main__":
    # gen_keypoints_for_video("/home/gokulnc/data-disk/datasets/Chinese/CSL/word/color/000/P01_01_00_0._color.mp4", "sample.pkl")
    n_cores = multiprocessing.cpu_count()

    DIR = "isign_subset/"
    SAVE_DIR = "isign_holistic_poses/"

    os.makedirs(SAVE_DIR, exist_ok=True)

    file_paths = []
    save_paths = []
    for file in os.listdir(DIR):
        if file.endswith(".mp4"):  # Process all .mp4 files
            file_paths.append(os.path.join(DIR, file))
            save_paths.append(os.path.join(SAVE_DIR, file.replace(".mp4", "")))

    print("Folder created")
    
    Parallel(n_jobs=n_cores, backend="loky")(
        delayed(gen_keypoints_for_video)(path, save_path)
        for path, save_path in tqdm(zip(file_paths, save_paths))
    )


Folder created
Loaded 92 frames from isign_subset/bMqwQWqdZJ8--1.mp4
Loaded 101 frames from isign_subset/nLG-B3e-pP0--40.mp4
Loaded 104 frames from isign_subset/ZZ7QTi7zYDw--0.mp4
Loaded 96 frames from isign_subset/UbSkIyCl70A--181.mp4
Loaded 109 frames from isign_subset/366805e50016-40.mp4
Loaded 109 frames from isign_subset/9DdtGulGsNM--15.mp4
Loaded 100 frames from isign_subset/Y1GBhK3p6HU--2.mp4
Loaded 183 frames from isign_subset/nnIo6Q41SY0_w.mp4
Loaded 179 frames from isign_subset/NoVPRWXWUZE--192.mp4
Loaded 132 frames from isign_subset/eChuoBjctPk--15.mp4
Loaded 237 frames from isign_subset/zE5S6z5wvwc--15.mp4
Loaded 125 frames from isign_subset/LTbn1Gekuk4--22.mp4


I0000 00:00:1727957233.096381 4076783 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727957233.293270 4077332 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: NVIDIA GeForce GT 710/PCIe/SSE2
I0000 00:00:1727957233.297917 4076786 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727957233.298187 4076787 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5


Loaded 331 frames from isign_subset/ouaMiZ5aaco--4.mp4
Loaded 313 frames from isign_subset/MR4XQ2x5o48--42.mp4
Loaded 352 frames from isign_subset/Kb22VWQxt3s--0.mp4


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1727957233.370512 4076780 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727957233.519582 4076778 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
W0000 00:00:1727957233.530099 4077317 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
I0000 00:00:1727957233.536267 4076779 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727957233.595059 4077382 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: NVIDIA GeForce GT 710/PCIe/SSE2
I0000 00:00:1727957233.596073 4076791 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
I0000 00:00:1727957233.679144 4076792 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00

Loaded 855 frames from isign_subset/jv_sRQ9NtAo--7.mp4


...

Loaded 133 frames from isign_subset/w9VwLKQIcus--6.mp4


I0000 00:00:1727961956.192070 4114915 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727961956.393668 4119726 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 470.256.02), renderer: NVIDIA GeForce GT 710/PCIe/SSE2
W0000 00:00:1727961956.745987 4119711 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Loaded 168 frames from isign_subset/lXSHkJsuw2s--22.mp4


W0000 00:00:1727961957.954374 4119707 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727961957.970152 4119711 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727961957.980672 4119710 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727961958.002176 4119707 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727961958.071740 4119719 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727961958.204050 4119708 inference_feedback_manager.cc:114] Feedback manager 

KeyboardInterrupt: 